In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import cartoframes

In [ ]:
full_gdf = pd.DataFrame(columns=['geometry', 'raster_val'])

for year in range(2002,2019):

    mask = None

    path = f'../images/NASA/BurnedArea{year}.tif'

    src = rasterio.open(path)

    image = src.read(1)

    results = (
        {'properties': {'raster_val': v}, 'geometry': s}

        for i, (s, v) in enumerate(
            features.shapes(image, mask=mask, transform=src.transform)))

    geoms = list(results)

    gdf = gpd.GeoDataFrame.from_features(geoms)
    
    gdf['year'] = year
    
    gdf.crs = 'epsg:4326'

    #gdf.crs = 'epsg:32630'

    #gdf = gdf.to_crs('epsg:4326')

    gdf = gdf[gdf['raster_val'] > 1]
    
    full_gdf = full_gdf.append(gdf, ignore_index=True)

In [ ]:
from cartoframes.viz import Map, Layer

In [ ]:
cartoframes.auth.set_default_credentials('../../Desktop/carto_creds.json')

In [ ]:
cartoframes.to_carto(full_gdf, 'results_nasa', if_exists='replace')

In [ ]:
from cartoframes.data.clients import SQLClient

SQLClient().execute("""UPDATE results_nasa SET the_geom = ST_MakeValid(the_geom), the_geom_webmercator = ST_MakeValid(the_geom_webmercator)""")

In [ ]:
from cartoframes.viz import color_bins_style, basemaps, popup_element

layer_acfcd = Layer(
    """SELECT acfcd.cartodb_id, acfcd.the_geom, acfcd.the_geom_webmercator, acfcd.raster_val as year FROM results_acfcd acfcd, cntr_rg_01m_2020_4326 ctr WHERE acfcd.raster_val < 2018 AND ctr.CNTR_ID IN ('ES', 'PT') AND ST_Intersects(acfcd.the_geom, ctr.the_geom)""",
    color_bins_style('year', palette='TealRose', bins=5, opacity=0.7), title='ACFCD', default_popup_hover=False, default_popup_click=False)

layer_ocd = Layer(
    """SELECT ocd.cartodb_id, ocd.the_geom, ocd.the_geom_webmercator, ocd.raster_val as year FROM results_ocd ocd, cntr_rg_01m_2020_4326 ctr WHERE ocd.raster_val < 2018 AND ctr.CNTR_ID IN ('ES', 'PT') AND ST_Intersects(ocd.the_geom, ctr.the_geom)""",
    color_bins_style('year', palette='TealRose', bins=5, opacity=0.7), title='OCD', default_legend=False, default_popup_hover=False, default_popup_click=False, popup_click=[popup_element('year', title='Year of event')])

layer_nasa = Layer(
    """SELECT nasa.cartodb_id, nasa.the_geom, nasa.the_geom_webmercator, nasa.year as year FROM results_nasa nasa, cntr_rg_01m_2020_4326 ctr WHERE nasa.raster_val < 2018 AND ctr.CNTR_ID IN ('ES', 'PT') AND ST_Intersects(nasa.the_geom, ctr.the_geom)""",
    color_bins_style('year', palette='TealRose', bins=5, opacity=0.7), title='NASA', default_legend=False, default_popup_hover=False, default_popup_click=False)


my_map = Map([layer_acfcd, layer_ocd, layer_nasa],
            basemap=basemaps.voyager,
            layer_selector=True
           )

In [ ]:
my_map

In [ ]:
my_map.publish('results_ocd', password=None, if_exists='replace')

In [ ]:
gdf_nasa = cartoframes.read_carto("""SELECT nasa.cartodb_id, nasa.the_geom, nasa.the_geom_webmercator, nasa.year as year FROM results_nasa nasa, cntr_rg_01m_2020_4326 ctr WHERE nasa.raster_val < 2018 AND ctr.CNTR_ID IN ('ES', 'PT') AND ST_Intersects(nasa.the_geom, ctr.the_geom)""")

In [ ]:
gdf_nasa.head()

In [ ]:
gdf_nasa['year'] = gdf_nasa['year'].astype(int)

In [ ]:
gdf_nasa.to_file('modelo_nasa.geojson', driver='GeoJSON')

In [6]:
model = 'modelo_nasa'

path = f'../results/vector/{model}.geojson'

gdf = gpd.read_file(path)

gdf.drop('cartodb_id', axis=1, inplace=True)

gdf.to_file(path, driver='GeoJSON')

../results/vector/modelo_nasa.geojson: No such file or directory
driver GeoJSON does not support creation option ENCODING


In [7]:
gdf = gpd.read_file(path)

In [8]:
gdf.head()

,year,geometry
0,2002,"POLYGON ((-6.33312 43.49643, -6.33312 43.48744..."
1,2002,"POLYGON ((-5.57854 43.47846, -5.57854 43.46948..."
2,2002,"POLYGON ((-2.75783 43.45151, -2.75783 43.44253..."
3,2002,"POLYGON ((-5.74922 43.42456, -5.74922 43.41558..."
4,2002,"POLYGON ((-5.74023 43.42456, -5.74023 43.41558..."


In [9]:
cartoframes.auth.set_default_credentials(username='mmoncadaisla', api_key='3c1aed5c809da52fdcfbd067ac0c6de5d91ad5d7')

In [ ]:
from cartoframes.viz import color_bins_style, basemaps, popup_element

layer_acfcd = Layer(
    '',
    color_bins_style('year', palette='TealRose', bins=5, opacity=0.7), title='ACFCD', default_popup_hover=False, default_popup_click=False)

layer_ocd = Layer(
    '',
    color_bins_style('year', palette='TealRose', bins=5, opacity=0.7), title='OCD', default_legend=False, default_popup_hover=False, default_popup_click=False, popup_click=[popup_element('year', title='Year of event')])

layer_nasa = Layer(
    '',
    color_bins_style('year', palette='TealRose', bins=5, opacity=0.7), title='NASA', default_legend=False, default_popup_hover=False, default_popup_click=False)


my_map = Map([layer_acfcd, layer_ocd, layer_nasa],
            basemap=basemaps.voyager,
            layer_selector=True
           )